In [33]:
import http.client
import json
from settings import API_KEY
import time
from pandas.io.json import json_normalize
import pandas as pd
import  csv
import ast

## Funções

In [49]:
def get_url_credits(movie_id, api_key):
    return("/3/movie/" + str(movie_id) + "/credits" + "?api_key=" + str(api_key))

def get_json(conn, url):
	payload = "{}"
	conn.request("GET", url, payload)
	res = conn.getresponse()
	data = res.read()
	return(json.loads(data.decode("    utf-8")))

In [47]:
def get_movie_credits(ids):
    
	conn = http.client.HTTPSConnection("api.themoviedb.org")    
	movie_credits_df = pd.DataFrame()
    
	for movie_id in ids:
		url = get_url_credits(movie_id, API_KEY)
        
		try:
			credits_data = get_json(conn, url)
		except Exception as e:
			if (credits_data['status_code'] == 25):
				time.sleep(10)                
				credits_data = get_json(conn, url)        
        
		df = json_normalize(credits_data)
		movie_credits_df = movie_credits_df.append(df)

	return movie_credits_df

In [29]:
def get_cast(ids, movie_credits_df):

    movie_cast_df = pd.DataFrame()

    for idx,movie_id in enumerate(ids,40):

        movie_item = pd.DataFrame()
        movie_item = movie_credits_df[movie_credits_df['id']==movie_id].reset_index()
        print("%s of %s" % (idx,len(ids)))

        print(movie_item['cast'])
        
        for cast_id in movie_item['cast'][0]:
            cast_id.update({'movie_id': int(movie_id)})
            cast_id.pop('profile_path', None)
            movie_cast_df = movie_cast_df.append(cast_id, ignore_index=True)
    
    return movie_cast_df

## Testes

In [43]:
with open('out_sample.csv', 'r') as f:
#with open('out_all.csv', 'r') as f:    
    reader = csv.reader(f, skipinitialspace=True, delimiter=',')
    ids_list = list(reader)
    
ids = [item for sublist in ids_list for item in sublist]

In [44]:
len(ids)

156

In [45]:
# Transformando ids em numérico
ids_num = list(map(int, ids))

In [50]:
movie_credits_df = get_movie_credits(ids_num)

In [52]:
movie_credits_df.shape

(156, 5)

In [53]:
movie_credits_df = movie_credits_df.drop_duplicates('id', keep='first')

In [55]:
movie_credits_df['id'].isnull().sum()

1

In [56]:
movie_credits_df[movie_credits_df['id'].isnull()]

,cast,crew,id,status_code,status_message
0,NaN,NaN,NaN,25.0,Your request count (41) is over the allowed li...


In [57]:
movie_credits_df = movie_credits_df[movie_credits_df['status_code'] != 25]

In [58]:
movie_credits_df['id'].isnull().sum()

0

In [71]:
movie_credits_df = movie_credits_df.reset_index()

In [72]:
movie_cast_df = get_cast(ids_num, movie_credits_df)

40 of 156
0    [{'id': 529, 'movie_id': 77, 'cast_id': 4, 'ge...
Name: cast, dtype: object
41 of 156
0    [{'id': 934, 'movie_id': 98, 'cast_id': 8, 'ge...
Name: cast, dtype: object
42 of 156
0    [{'id': 3894, 'movie_id': 1359, 'cast_id': 3, ...
Name: cast, dtype: object
43 of 156
0    [{'id': 31, 'movie_id': 8358, 'cast_id': 3, 'g...
Name: cast, dtype: object
44 of 156
0    [{'id': 2461, 'movie_id': 7443, 'cast_id': 3, ...
Name: cast, dtype: object
45 of 156
0    [{'id': 500, 'movie_id': 955, 'cast_id': 31, '...
Name: cast, dtype: object
46 of 156
0    [{'id': 1121, 'movie_id': 107, 'cast_id': 19, ...
Name: cast, dtype: object
47 of 156
0    [{'id': 1461, 'movie_id': 2133, 'cast_id': 16,...
Name: cast, dtype: object
48 of 156
0    [{'id': 15555, 'movie_id': 6282, 'cast_id': 1,...
Name: cast, dtype: object
49 of 156
0    [{'id': 8600, 'movie_id': 40096, 'cast_id': 1,...
Name: cast, dtype: object
50 of 156
0    [{'id': 1619, 'movie_id': 146, 'cast_id': 25, ...
Name: cast, dtype: object

IndexError: index out of bounds

In [63]:
ids_num[80]

10384

In [65]:
movie_credits_df.iloc[80]

cast              [{'id': 112, 'cast_id': 1, 'gender': 1, 'credi...
crew              [{'id': 7623, 'profile_path': '/uEAaCUJzyMI8N5...
id                                                             2046
status_code                                                     NaN
status_message                                                  NaN
Name: 0, dtype: object

In [64]:
movie_credits_df[movie_credits_df['id']==10384]

,cast,crew,id,status_code,status_message
0,"[{'id': 13548, 'cast_id': 6, 'gender': 2, 'cre...","[{'id': 1723, 'profile_path': '/gCU3dgnNaZuvmA...",10384.0,NaN,NaN


In [74]:
movie_credits_df.head(82)

,index,cast,crew,id,status_code,status_message
0,0,"[{'id': 529, 'movie_id': 77, 'cast_id': 4, 'ge...","[{'id': 525, 'profile_path': '/7OGmfDF4VHLLgbj...",77.0,NaN,NaN
1,0,"[{'id': 934, 'movie_id': 98, 'cast_id': 8, 'ge...","[{'id': 578, 'profile_path': '/oTAL0z0vsjipCru...",98.0,NaN,NaN
2,0,"[{'id': 3894, 'movie_id': 1359, 'cast_id': 3, ...","[{'id': 16378, 'profile_path': '/qlWi0kufzB7lC...",1359.0,NaN,NaN
3,0,"[{'id': 31, 'movie_id': 8358, 'cast_id': 3, 'g...","[{'id': 24, 'profile_path': '/isCuZ9PWIOyXzdf3...",8358.0,NaN,NaN
4,0,"[{'id': 2461, 'movie_id': 7443, 'cast_id': 3, ...","[{'id': 2359, 'profile_path': None, 'job': 'Ex...",7443.0,NaN,NaN
5,0,"[{'id': 500, 'movie_id': 955, 'cast_id': 31, '...","[{'id': 122, 'profile_path': '/j4BGxQq5klTT7hi...",955.0,NaN,NaN
6,0,"[{'id': 1121, 'movie_id': 107, 'cast_id': 19, ...","[{'id': 29, 'profile_path': None, 'job': 'Exec...",107.0,NaN,NaN
7,0,"[{'id': 1461, 'movie_id': 2133, 'cast_id': 16,...","[{'id': 5231, 'profile_path': '/t3Fxy4SBeOWj8T...",2133.0,NaN,NaN
8,0,"[{'id': 15555, 'movie_id': 6282, 'cast_id': 1,...","[{'id': 770, 'profile_path': '/p2rd1kgq7GiCx2n...",6282.0,NaN,NaN
9,0,"[{'id': 8600, 'movie_id': 40096, 'cast_id': 1,...","[{'id': 102560, 'profile_path': '/tAjR4ocM6Vjj...",40096.0,NaN,NaN
